In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Загружаем модель для создания embedding-векторов
model = SentenceTransformer('all-MiniLM-L6-v2')

def cosine_similarity(text1, text2):
    # Преобразуем текст в embedding-векторы
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    
    # Вычисляем косинусное сходство
    cosine_sim = util.pytorch_cos_sim(embedding1, embedding2)
    
    return cosine_sim.item()  # Возвращаем значение сходства как float

# Пример данных
question = "Какую операционную систему поддерживает приложение?"
context = "Наше приложение поддерживает Windows и macOS."

# Вычисляем косинусное сходство
similarity_score = cosine_similarity(question, context)
print(f"Cosine Similarity: {similarity_score:.4f}")
